# 14.3 IT 运维智能助手

Multi-Agent 故障诊断系统。

**预估成本**: ~$0.20

In [ ]:
!pip install openai -q

In [ ]:
import os
import json
from datetime import datetime
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

## 运维助手实现

In [ ]:
class OpsAssistant:
    def __init__(self):
        self.client = OpenAI()
        self.kb = {
            'timeout': {'symptoms': 'connection timeout', 'solution': '增加连接池或优化查询'},
            'oom': {'symptoms': 'OutOfMemoryError', 'solution': '分析堆转储或增加内存'}
        }
    
    def analyze_logs(self, logs):
        prompt = f'''分析日志，提取错误信息。

日志：
{logs}

输出JSON：
{{
  "error_type": "<类型>",
  "error_message": "<信息>",
  "affected_components": ["<组件>"],
  "error_count": <次数>
}}'''
        
        r = self.client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role': 'user', 'content': prompt}],
            response_format={'type': 'json_object'}
        )
        return json.loads(r.choices[0].message.content)
    
    def diagnose(self, analysis):
        similar = [v for k, v in self.kb.items() if k in analysis['error_type'].lower()]
        prompt = f'''诊断故障根本原因。

日志分析：{json.dumps(analysis, ensure_ascii=False)}
历史案例：{json.dumps(similar, ensure_ascii=False)}

输出JSON：
{{
  "root_cause": "<原因>",
  "severity": "high|medium|low",
  "confidence": <0-100>
}}'''
        
        r = self.client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'user', 'content': prompt}],
            response_format={'type': 'json_object'}
        )
        return json.loads(r.choices[0].message.content)
    
    def recommend_solution(self, diagnosis):
        prompt = f'''推荐解决方案。

诊断：{json.dumps(diagnosis, ensure_ascii=False)}

输出JSON：
{{
  "solutions": [
    {{
      "priority": 1,
      "description": "<方案>",
      "steps": ["<步骤>"],
      "estimated_time": "<时间>"
    }}
  ]
}}'''
        
        r = self.client.chat.completions.create(
            model='gpt-4o',
            messages=[{'role': 'user', 'content': prompt}],
            response_format={'type': 'json_object'}
        )
        return json.loads(r.choices[0].message.content)
    
    def handle_incident(self, alert):
        print('\n🚨 运维助手启动')
        print('[1/3] 分析日志...')
        analysis = self.analyze_logs(alert['logs'])
        print(f"✓ 错误类型: {analysis['error_type']}")
        
        print('[2/3] 诊断原因...')
        diagnosis = self.diagnose(analysis)
        print(f"✓ 根本原因: {diagnosis['root_cause']}")
        
        print('[3/3] 生成方案...')
        solutions = self.recommend_solution(diagnosis)
        print(f"✓ 已生成 {len(solutions['solutions'])} 个方案")
        
        return {
            'alert': alert,
            'analysis': analysis,
            'diagnosis': diagnosis,
            'solutions': solutions,
            'timestamp': datetime.now().isoformat()
        }

# 测试
assistant = OpsAssistant()

alert = {
    'timestamp': '2026-02-20 03:15:30',
    'service': 'api-server',
    'logs': '''[03:15:28] ERROR: Database connection timeout after 5000ms
[03:15:28] ERROR: Connection pool exhausted (50/50 connections)
[03:15:29] ERROR: Retry failed, returning 504'''
}

report = assistant.handle_incident(alert)
print(f"\n📋 诊断报告：")
print(json.dumps(report, indent=2, ensure_ascii=False))